In [1]:
import pandas as pd
import requests

In [2]:
admis = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv',
                    infer_datetime_format=True, parse_dates=[8,9],
                   dtype= {6:'str',12:'str',14:'str',15:'str'})
visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv',
                     infer_datetime_format=True, parse_dates=[12],
                    dtype={8:'str',9:'str',10:'str',11:'str'})

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
admis.iloc[:,6:8]

,Attending Provider NPI,Facility Provider NPI
0,1417170523,1306890389
1,1669599197,1902803315
2,1407880511,1902803315
3,1407008907,1427055839
4,1518045608,1992818256
...,...,...
6208,1720043417,1689772592
6209,1215968474,1427055839
6210,1861831836,1689608150
6211,1750544516,1689608150


In [4]:
visits.iloc[:,8:12]

,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI
0,NaN,NaN,1487856779,1043427248
1,1528030996,1902803315,1528030996,1043427248
2,NaN,NaN,1477579365,1043427248
3,1821178088,1104842020,1821178088,1407058787
4,NaN,NaN,1609004191,1184722779
...,...,...,...,...
231368,NaN,NaN,1851738108,1184722779
231369,NaN,NaN,1215163035,1043427248
231370,NaN,NaN,1215163035,1043427248
231371,NaN,NaN,1144252396,1962790014


In [5]:
npi = admis.iloc[:,6].append((admis.iloc[:,7], visits.iloc[:,8],
                              visits.iloc[:,9],visits.iloc[:,10],
                              visits.iloc[:,11]), ignore_index=True)

In [6]:
npi = npi.drop_duplicates().reset_index().dropna().rename(columns={0:'NPI'}).drop(columns=['index'])
npi

,NPI
0,1417170523
1,1669599197
2,1407880511
3,1407008907
4,1518045608
...,...
5411,1336222397
5412,1336150572
5413,1417017435
5414,1790964864


In [7]:
npi.iloc[2,0]

'1407880511'

In [8]:
endpoint = 'https://npiregistry.cms.hhs.gov/api/?version=2.0'
params = {'number' : '1407880511'}

In [9]:
response = requests.get(endpoint , params = params)

In [10]:
response

<Response [200]>

In [11]:
res = response.json()
res

{'result_count': 1,
 'results': [{'enumeration_type': 'NPI-1',
   'number': 1407880511,
   'last_updated_epoch': 1578914062,
   'created_epoch': 1152576000,
   'basic': {'first_name': 'CAROLYN',
    'last_name': 'SACHS',
    'middle_name': 'JOY',
    'credential': 'MD',
    'sole_proprietor': 'NO',
    'gender': 'F',
    'enumeration_date': '2006-07-11',
    'last_updated': '2020-01-13',
    'status': 'A',
    'name': 'SACHS CAROLYN',
    'certification_date': '2020-01-13'},
   'other_names': [],
   'addresses': [{'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'LOCATION',
     'address_type': 'DOM',
     'address_1': '757 WESTWOOD PLZ',
     'address_2': 'RM 14-19',
     'city': 'LOS ANGELES',
     'state': 'CA',
     'postal_code': '900953075',
     'telephone_number': '310-825-2111'},
    {'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'MAILING',
     'address_type': 'DOM',
     'address_1': '5767 W CENTURY BLV

In [12]:
res.keys()

dict_keys(['result_count', 'results'])

In [13]:
res['results'][0]

{'enumeration_type': 'NPI-1',
 'number': 1407880511,
 'last_updated_epoch': 1578914062,
 'created_epoch': 1152576000,
 'basic': {'first_name': 'CAROLYN',
  'last_name': 'SACHS',
  'middle_name': 'JOY',
  'credential': 'MD',
  'sole_proprietor': 'NO',
  'gender': 'F',
  'enumeration_date': '2006-07-11',
  'last_updated': '2020-01-13',
  'status': 'A',
  'name': 'SACHS CAROLYN',
  'certification_date': '2020-01-13'},
 'other_names': [],
 'addresses': [{'country_code': 'US',
   'country_name': 'United States',
   'address_purpose': 'LOCATION',
   'address_type': 'DOM',
   'address_1': '757 WESTWOOD PLZ',
   'address_2': 'RM 14-19',
   'city': 'LOS ANGELES',
   'state': 'CA',
   'postal_code': '900953075',
   'telephone_number': '310-825-2111'},
  {'country_code': 'US',
   'country_name': 'United States',
   'address_purpose': 'MAILING',
   'address_type': 'DOM',
   'address_1': '5767 W CENTURY BLVD',
   'address_2': 'SUITE 400',
   'city': 'LOS ANGELES',
   'state': 'CA',
   'postal_code'

In [14]:
res['results'][0].keys()

dict_keys(['enumeration_type', 'number', 'last_updated_epoch', 'created_epoch', 'basic', 'other_names', 'addresses', 'taxonomies', 'identifiers'])

In [15]:
res['results'][0]['enumeration_type']

'NPI-1'

In [16]:
endpoint = 'https://npiregistry.cms.hhs.gov/api/?version=2.0'
enumeration = list()
entry = []
for x in range(0,len(npi)):
    params = {'number' : npi.iloc[x,0]}
    print(str(x) + str(params))
    response = requests.get(endpoint , params = params)
    try:
        res = response.json()
        enumeration.append(res['results'][0]['enumeration_type'])
        entry.append(res['results'][0])
    except:
        enumeration.append('NPI-0')
        entry.append('Error')

0{'number': '1417170523'}
1{'number': '1669599197'}
2{'number': '1407880511'}
3{'number': '1407008907'}
4{'number': '1518045608'}
5{'number': '1346556891'}
6{'number': '1316989247'}
7{'number': '1417995317'}
8{'number': '1396818134'}
9{'number': '1447495387'}
10{'number': '1528163003'}
11{'number': '1366604969'}
12{'number': '1427375757'}
13{'number': '1619073954'}
14{'number': '1972526721'}
15{'number': '1366847147'}
16{'number': '1750346474'}
17{'number': '1932127248'}
18{'number': '1346566221'}
19{'number': '1598993990'}
20{'number': '1720240526'}
21{'number': '1023149044'}
22{'number': '1508841602'}
23{'number': '1730127069'}
24{'number': '1053334292'}
25{'number': '1922323856'}
26{'number': '1700828704'}
27{'number': '1982865044'}
28{'number': '1902850498'}
29{'number': '1710270442'}
30{'number': '1427154533'}
31{'number': '1306902739'}
32{'number': '1518963032'}
33{'number': '1750425278'}
34{'number': '1427344951'}
35{'number': '1073566931'}
36{'number': '1679541775'}
37{'number'

297{'number': '1922328590'}
298{'number': '1588624522'}
299{'number': '1023036068'}
300{'number': '1023029162'}
301{'number': '1154683043'}
302{'number': '1144301722'}
303{'number': '1851554372'}
304{'number': '1215961628'}
305{'number': '1871579458'}
306{'number': '1013973783'}
307{'number': '1609140524'}
308{'number': '1619040219'}
309{'number': '1851374441'}
310{'number': '1215183066'}
311{'number': '1730408774'}
312{'number': '1669570271'}
313{'number': '1326006941'}
314{'number': '1316011265'}
315{'number': '1104087808'}
316{'number': '1780614362'}
317{'number': '1316160211'}
318{'number': '1386651867'}
319{'number': '1487688198'}
320{'number': '1558310557'}
321{'number': '1295977197'}
322{'number': '1285750943'}
323{'number': '1760631287'}
324{'number': '1790966877'}
325{'number': '1548440902'}
326{'number': '1275544876'}
327{'number': '1073547808'}
328{'number': '1518999069'}
329{'number': '1245477652'}
330{'number': '1053331199'}
331{'number': '1588779870'}
332{'number': '19524

590{'number': '1033352638'}
591{'number': '1396828901'}
592{'number': '1487635298'}
593{'number': '1679508923'}
594{'number': '1023268182'}
595{'number': '1831279991'}
596{'number': '1497785208'}
597{'number': '1154536845'}
598{'number': '1053352302'}
599{'number': '1083971501'}
600{'number': '1407831530'}
601{'number': '1124182084'}
602{'number': '1700832201'}
603{'number': '1417986811'}
604{'number': '1568660165'}
605{'number': '1417951997'}
606{'number': '1598993248'}
607{'number': '1184607921'}
608{'number': '1326136185'}
609{'number': '1508807462'}
610{'number': '1730198128'}
611{'number': '1063647865'}
612{'number': '1265588164'}
613{'number': '1437163334'}
614{'number': '1184915001'}
615{'number': '1780901231'}
616{'number': '1619136838'}
617{'number': '1457309775'}
618{'number': '1114981594'}
619{'number': '1730335191'}
620{'number': '1366646952'}
621{'number': '1073733861'}
622{'number': '1245551175'}
623{'number': '1295780260'}
624{'number': '1992140073'}
625{'number': '14270

883{'number': '1285659482'}
884{'number': '1376515023'}
885{'number': '1801236401'}
886{'number': '1851311930'}
887{'number': '1255351854'}
888{'number': '1972528305'}
889{'number': '1013114719'}
890{'number': '1619048659'}
891{'number': '1427175827'}
892{'number': '1033361282'}
893{'number': '1861620593'}
894{'number': '1336184670'}
895{'number': '1770810590'}
896{'number': '1326134859'}
897{'number': '1497827562'}
898{'number': '1932121498'}
899{'number': '1841272150'}
900{'number': '1831144583'}
901{'number': '1730316969'}
902{'number': '1750492146'}
903{'number': '1538222310'}
904{'number': '1700047362'}
905{'number': '1194800094'}
906{'number': '1750371142'}
907{'number': '1205271285'}
908{'number': '1184828444'}
909{'number': '1417101668'}
910{'number': '1205190659'}
911{'number': '1164565776'}
912{'number': '1376641258'}
913{'number': '1508943432'}
914{'number': '1356372791'}
915{'number': '1508828179'}
916{'number': '1790050979'}
917{'number': '1386625721'}
918{'number': '10533

1170{'number': '1649235300'}
1171{'number': '1023151735'}
1172{'number': '1851617641'}
1173{'number': '1902067325'}
1174{'number': '1497980932'}
1175{'number': '1902833080'}
1176{'number': '1134232051'}
1177{'number': '1831175942'}
1178{'number': '1023245164'}
1179{'number': '1043379092'}
1180{'number': '1619311669'}
1181{'number': '1194160242'}
1182{'number': '1306000922'}
1183{'number': '1588898845'}
1184{'number': '1093767337'}
1185{'number': '1164450714'}
1186{'number': '1992913222'}
1187{'number': '1043338734'}
1188{'number': '1386683621'}
1189{'number': '1265574867'}
1190{'number': '1063402477'}
1191{'number': '1184682379'}
1192{'number': '1316967938'}
1193{'number': '1891956678'}
1194{'number': '1659442317'}
1195{'number': '1871631051'}
1196{'number': '1245566264'}
1197{'number': '1649505009'}
1198{'number': '1508165077'}
1199{'number': '1083654529'}
1200{'number': '1043218209'}
1201{'number': '1174565451'}
1202{'number': '1982939955'}
1203{'number': '1538170808'}
1204{'number':

1453{'number': '1538117981'}
1454{'number': '1205944485'}
1455{'number': '1861405029'}
1456{'number': '1235391418'}
1457{'number': '1750650784'}
1458{'number': '1144486929'}
1459{'number': '1437485968'}
1460{'number': '1669497996'}
1461{'number': '1225154727'}
1462{'number': '1487940995'}
1463{'number': '1013996529'}
1464{'number': '1003230079'}
1465{'number': '1942291729'}
1466{'number': '1316216617'}
1467{'number': '1912235482'}
1468{'number': '1821037342'}
1469{'number': '1184686057'}
1470{'number': '1174883995'}
1471{'number': '1851381081'}
1472{'number': '1689610990'}
1473{'number': '1255411518'}
1474{'number': '1093050858'}
1475{'number': '1588654693'}
1476{'number': '1487611372'}
1477{'number': '1467643700'}
1478{'number': '1932297253'}
1479{'number': '1306959135'}
1480{'number': '1710265244'}
1481{'number': '1245469246'}
1482{'number': '1346306008'}
1483{'number': '1578587739'}
1484{'number': '1285671016'}
1485{'number': '1528196185'}
1486{'number': '1790055705'}
1487{'number':

1736{'number': '1932297330'}
1737{'number': '1720077480'}
1738{'number': '1700143849'}
1739{'number': '1538484084'}
1740{'number': '1376770768'}
1741{'number': '1912081654'}
1742{'number': '1689693582'}
1743{'number': '1821215302'}
1744{'number': '1588933261'}
1745{'number': '1063754448'}
1746{'number': '1962794701'}
1747{'number': '1740388891'}
1748{'number': '1740589290'}
1749{'number': '1174537575'}
1750{'number': '1508849035'}
1751{'number': '1295962025'}
1752{'number': '1255778619'}
1753{'number': '1497996649'}
1754{'number': '1003995143'}
1755{'number': '1114925385'}
1756{'number': '1922274570'}
1757{'number': '1770548372'}
1758{'number': '1033439906'}
1759{'number': '1922267319'}
1760{'number': '1336197730'}
1761{'number': '1033144985'}
1762{'number': '1629230495'}
1763{'number': '1104812643'}
1764{'number': '1942316807'}
1765{'number': '1821361700'}
1766{'number': '1215103023'}
1767{'number': '1184611816'}
1768{'number': '1376632547'}
1769{'number': '1215189683'}
1770{'number':

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
enumeration

In [ ]:
entry

In [ ]:
npidf = pd.DataFrame()
npidf['npi'] = npi.iloc[:,0]

In [ ]:
npidf['enumeration'] = enumeration

In [ ]:
npidf['entry'] = entry

In [ ]:
npidf

In [ ]:
npidf.to_csv('../output/npi.csv')